In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# word2vec을 활용한 리뷰 감정 예측 모델

In [2]:
import pandas as pd
from gensim.models import KeyedVectors
# 전처리가 완료된 데이터
tokenized_data = pd.read_pickle(r'/content/drive/MyDrive/8_DL/tokenized_data.pkl')
label_data = pd.read_pickle(r'/content/drive/MyDrive/8_DL/labels.pkl')
loaded_model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/8_DL/Word2Vec')

In [ ]:
# # 이렇게 해도 됨

# from google.colab import drive
# drive.mount('/content/drive')

# # 개인의 구글 드라이브 폴더 path를 넣어줘야 합니다.
# dir_path = '/content/drive/MyDrive/lecture/진행 중/서울산업진흥원(SBA)/02. 딥러닝/05. NLP/data'

# import os
# os.chdir(dir_path) # 현재 경로 수정

# import pandas as pd
# from gensim.models import KeyedVectors
# # 전처리가 완료된 데이터
# tokenized_data = pd.read_pickle(r'tokenized_data.pkl')
# label_data = pd.read_pickle(r'labels.pkl')
# loaded_model = KeyedVectors.load_word2vec_format('Word2Vec')

In [3]:
tokenized_data[0] # 얘들을 평균의 벡터로 보겠다

['더빙', '진짜', '짜증나다', '목소리']

In [4]:
# sentence는 여러개의 word로 되어 있어서 하나의 sentence vector를 word 벡터들의 평균으로 지정
loaded_model[tokenized_data[0]].mean(axis = 0)

array([ 8.9702332e-01, -6.6159070e-02,  3.6638540e-01, -8.1567115e-01,
       -1.7353792e-01, -5.0399268e-01,  6.6600427e-02,  2.1103202e-01,
        2.2228357e-01,  5.1702619e-01, -7.3621762e-01, -3.2955855e-01,
       -7.0753258e-01, -9.8549828e-02,  2.0006344e+00, -6.6509444e-01,
       -5.8446419e-01,  7.1924657e-01,  6.7536491e-01, -9.4602478e-01,
       -8.7658286e-02, -2.2775224e-01, -2.9901631e-02,  4.6559694e-01,
        9.6563715e-01, -1.4495753e-01, -7.0928502e-01, -3.1341937e-01,
       -3.7604558e-01, -6.7241508e-01, -8.7653261e-01, -2.3406097e-01,
       -4.4607145e-01,  1.6622618e-02, -1.1627940e+00, -3.0496413e-01,
        8.6092311e-01,  2.5329712e-01, -7.6038420e-01, -5.1928595e-02,
       -1.5346073e-01, -6.4136028e-02, -6.9277287e-03,  9.4044879e-02,
       -2.8240386e-01,  6.8737440e-02, -4.3235993e-01,  6.8819714e-01,
       -3.9162856e-01, -8.2110927e-02, -4.5644337e-01,  1.0458933e+00,
        4.8411173e-01,  6.4807987e-01, -4.2266831e-01,  2.3603466e-01,
      

In [5]:
from tqdm import tqdm
import numpy as np

data_vectors = []
embedding_size = 100
for tokened_sentence in tqdm(tokenized_data): # 토큰화된 센텐스가 들어옴
  try:
    senetence_vector = loaded_model[tokened_sentence].mean(axis = 0)
    data_vectors.append(senetence_vector)
  except:
    zeros_vector = np.zeros(embedding_size)
    data_vectors.append(zeros_vector)

100%|██████████| 199992/199992 [00:08<00:00, 24436.24it/s]


In [6]:
vector_data = np.array(data_vectors) # 데이터가 숫자로 바뀜. 이제 똑같이 모델 생성, 학습해주면됨!

In [7]:
vector_data.shape

(199992, 100)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vector_data, label_data, test_size = 0.25)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(149994, 100)
(49998, 100)
(149994,)
(49998,)


In [9]:
from sklearn.linear_model import LogisticRegression # 분류 모델
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [10]:
lr_model.score(X_train, y_train)

0.7243623078256464

In [11]:
lr_model.score(X_test, y_test)

0.7276891075643026

In [32]:
# randomforest

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0, n_estimators=100) # max_depth 안주니까 성능올라감
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [33]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}

In [34]:
rf.score(X_train, y_train)

0.8961825139672254

In [35]:
rf.score(X_test, y_test)

0.7393295731829274

In [37]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 42.5 MB/s eta 0:00:00


In [38]:
# 전처리 > 모델 활용해서 벡터로 바꿔주기 

from tqdm import tqdm
from konlpy.tag import Okt
import re

def predict_text(text):
  # 데이터 전처리
  stopwords = ['의', '를', '와', '과', '으로', '것', '잘', '되', '주', '보', '수',
              '좀', '이', '있', '은', '한', '아', '도', '에', '하', '는', '자',
              '하다', '가', '들', '걍', '등']

  text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', text)

  okt = Okt()
  tokenized_data = []

  tokenized_sentence = okt.morphs(text, stem = True)
  stopwords_removed_sentences = [token for token in tokenized_sentence if not token in stopwords]
  tokenized_data.append(stopwords_removed_sentences)
  
  # 데이터 벡터화
  data_vectors = []
  embedding_size = 100
  for tokened_sentence in tqdm(tokenized_data):
    try:
      sentence_vector = loaded_model[tokened_sentence].mean(axis = 0)
      data_vectors.append(sentence_vector)
    except:
      zeros_vector = np.zeros(embedding_size)
      data_vectors.append(zeros_vector)
  data_vectors = np.array(data_vectors)
  
  # 예측 및 결과 정리
  prediction = rf.predict(data_vectors)[0]
  if prediction == 1:
    answer = '긍정 리뷰입니다.'
  else:
    answer = '부정 리뷰입니다.'

  return answer

In [39]:
predict_text('노잼 노잼 노잼')

100%|██████████| 1/1 [00:00<00:00, 598.67it/s]


'부정 리뷰입니다.'

### 예측하기(강의 전)

In [ ]:
!pip install konlpy